In [1]:
import json
import time

In [2]:
with open("train.json") as training:
    training = json.loads(training.read())
    
with open("test.json") as testing:
    testing = json.loads(testing.read())

In [3]:
training_dict = {}
testing_dict = {}

for user in training.keys():
    for [item, rating] in training[user]:
        if user in training_dict:
            training_dict[user][item] = rating
        else:
            training_dict[user] = {item: rating}
            
for user in testing.keys():
    for [item, rating] in testing[user]:
        if user in testing_dict:
            testing_dict[user][item] = rating
        else:
            testing_dict[user] = {item: rating}

# print(training_dict['A12R54MKO17TW0'])

In [4]:
def sub_row_mean(ratings_dict):
    sub_row_mean_ratings_dict = {}
    for user in ratings_dict:
        list_item = list(ratings_dict[user].values())
        avg = sum(list_item) / len(list_item)
        for item in ratings_dict[user].keys():
            if user in sub_row_mean_ratings_dict:
                sub_row_mean_ratings_dict[user][item] = ratings_dict[user][item] - avg
            else:
                sub_row_mean_ratings_dict[user] = {item: ratings_dict[user][item] - avg}
    
    return sub_row_mean_ratings_dict
        
# training_dict2 = sub_row_mean(training_dict)
# print(training_dict2['A12R54MKO17TW0'])
# print(training_dict['A12R54MKO17TW0'])

In [5]:
import numpy as np
from numpy.linalg import norm
import math

def cosine_sim(ratings_dict, user):
    cosine_sim_val = {}

    for other_user in ratings_dict.keys():
        dot_prod = sum(ratings_dict[user][item]*ratings_dict[other_user].get(item, 0) for item in ratings_dict[user])
        user_norm = np.linalg.norm(np.array([rating for rating in ratings_dict[user].values()]))
        other_user_norm = np.linalg.norm(np.array([rating for rating in ratings_dict[other_user].values()]))
        norm_mul = user_norm * other_user_norm
        if norm_mul == 0:
            cos_sim = 0
        else:
            cos_sim = dot_prod / norm_mul
        if math.isnan(cos_sim):
            cos_sim = 0
        cosine_sim_val[other_user] = cos_sim
    
        if cosine_sim_val[other_user] > 1:
            cosine_sim_val[other_user] = 1
        if cosine_sim_val[other_user] < 0:
            
#             print('Sim < 0')
#             print('User: ' + str(user))
#             print('Other_user' + str(other_user))
#             print('Dot Prod: ' + str(dot_prod))
#             print('User Norm: ' + str(user_norm))
#             print('Other User Norm: ' + str(other_user_norm))
#             print('Norm_Mul: ' + str(norm_mul))
#             print('Sim value: ' + str(cosine_sim_val[other_user]))
            cosine_sim_val[other_user] = 0        
    
    return cosine_sim_val

In [6]:
# k = 2
# item = "B0048P1XXQ"
# user = "A12R54MKO17TW0"
# cos_sims = cosine_sim(training_dict2, 'A12R54MKO17TW0')

def predict_item_rating_avg(ratings_dict, cos_similarities_dict, k, item, user):
    # df.dropna(subset = [item])
    # df = df[df.reviewerID != user]
    # new_df =  df.nlargest(k, "cosine_sim")
    # return new_df[item].mean(axis = 0)
    cos_similarities_dict.pop(user)
    other_users = list(cos_similarities_dict.keys())
    for other_user in other_users:
        if item not in ratings_dict[other_user]:
            cos_similarities_dict.pop(other_user)
    
    list_cos_sim = sorted(cos_similarities_dict.items(), key=lambda x: x[1], reverse=True)[:k]
    #print(list_cos_sim)
    avg_rating = 0
    
    if len(list_cos_sim) != 0:
        for pair in list_cos_sim:
            avg_rating += ratings_dict[pair[0]][item]
        
        avg_rating = avg_rating / len(list_cos_sim)
    
    return avg_rating

# training_dict["AAE2DUEMTR30I"][item] = 4.0
# print(predict_item_rating_avg(training_dict, training_dict2, cos_sims, k, item, user))
# print(training_dict["AAE2DUEMTR30I"])
# print(training_dict["AQ6J4B5WLGRJ5"])
    
    

In [7]:
# k = 2
# item = "B0048P1XXQ"
# user = "A12R54MKO17TW0"
# cos_sims = cosine_sim(training_dict2, 'A12R54MKO17TW0')

def predict_item_rating_sim(ratings_dict, cos_similarities_dict, k, item, user):
    # df.dropna(subset = [item])
    # df = df[df.reviewerID != user]
    # new_df =  df.nlargest(k, "cosine_sim")
    # return (new_df["cosine_sim"].dot(new_df[item]))/(new_df["cosine_sim"].sum())
    if user in cos_similarities_dict:
        cos_similarities_dict.pop(user)
    other_users = list(cos_similarities_dict.keys())
    for other_user in other_users:
        if item not in ratings_dict[other_user]:
            cos_similarities_dict.pop(other_user)
    
    list_cos_sim = sorted(cos_similarities_dict.items(), key=lambda x: x[1], reverse=True)[:k]
    rating = 0
    similarities = 0
    if len(list_cos_sim) != 0:
        for pair in list_cos_sim:
            rating += ratings_dict[pair[0]][item] * cos_similarities_dict[pair[0]]
            similarities += cos_similarities_dict[pair[0]]
        
        if similarities == 0:
            total = 0
            rating = 0
            for rat in training_dict[user]:
                rating += training_dict[user][rat]
                total += 1
            rating = rating / total
        else:
            if rating / similarities > 5.1:
                print('Rating: ' + str(rating))
                print('Sim: ' + str(similarities))
                print('User: ' + str(user))
                print('Item: ' + str(item))
                for pair in list_cos_sim:

                    print("User: " + str(pair[0]))
                    print("Sim Score: " + str(cos_similarities_dict[pair[0]]))
                    print("Rating: " + str(ratings_dict[pair[0]][item]))
                    print('-----------------------------')
                    print('/n')
                    
            rating = rating / similarities
    if rating > 5.1:
        print("Flag")
        print("")
        rating = 4.99
    return rating

# training_dict["AAE2DUEMTR30I"][item] = 5.0
# print(predict_item_rating_sim(training_dict, training_dict2, cos_sims, k, item, user))
# print(training_dict["AAE2DUEMTR30I"])
# print(training_dict["AQ6J4B5WLGRJ5"])
    
    

In [8]:
def predict(ratings_dict, user, items):
    predicted_ratings_dict = {}
    sub_mean_dict = sub_row_mean(ratings_dict)
    cos_sim_dict = cosine_sim(sub_mean_dict, user)
    for item in items:
        avg_rating_prediction = predict_item_rating_avg(training_dict, cos_sim_dict.copy(), k, item, user)
        sim_rating_prediction = predict_item_rating_sim(training_dict, cos_sim_dict.copy(), k, item, user)
        
        predicted_ratings_dict[item] = (avg_rating_prediction, sim_rating_prediction)
        
    return predicted_ratings_dict

In [ ]:
k = 10
# item = ["B0048P1XXQ"]
# user = "A12R54MKO17TW0"

start = time.time()

    
N = 0
MAE_avg = 0
RMSE_avg = 0 
MAE_sim = 0
RMSE_sim = 0

i = 0
j = 0

for user, list_ratings in testing_dict.items():
    predicted_vals = predict(training_dict, user, list(testing_dict[user].keys()))
    orig_vals = testing_dict[user]
    i += 1

    #print(predicted_vals)
    #print(orig_vals)


    for item, (avg_rat, sim_rat) in predicted_vals.items():
        j += 1
        corr_rat = orig_vals[item]
        MAE_avg += abs(avg_rat-corr_rat)
        MAE_sim += abs(sim_rat-corr_rat)
        RMSE_avg += (avg_rat-corr_rat)**2
        RMSE_sim += (sim_rat-corr_rat)**2
        N+=1
        
        if j % 1000 == 0:
            print("Item" + str(j))
            print("User" + str(i))
            print(time.time() - start)
            print("predicted vals: ")
            print(predicted_vals)
            print("original vals: ")
            print(orig_vals)

MAE_avg = MAE_avg/N
MAE_sim = MAE_sim/N
RMSE_avg = math.sqrt(RMSE_avg/N)
RMSE_sim = math.sqrt(RMSE_sim/N)

print((MAE_avg, RMSE_avg))
print((MAE_sim, RMSE_sim))
    



end = time.time()

print(end - start) #=> 0.0011411383748054504

Item1000
User294
130.91967272758484
predicted vals: 
{'B001VW9UES': (4.3, 3.1009828382532656), 'B0042U8VUA': (5.0, 2.7628865979381443), 'B00138FSPU': (5.0, 2.7628865979381443), 'B001NTPENA': (5.0, 2.7628865979381443), 'B00W4SVHYE': (4.0, 2.7628865979381443), 'B000V61B74': (4.6, 4.0), 'B00122ULG8': (4.7, 2.0), 'B00124DTBK': (4.4, 5.0), 'B002WPXHV0': (4.8, 4.795690343361815), 'B00I83XD14': (4.714285714285714, 4.999999999999999), 'B00137MSX6': (4.7, 4.052554730217244), 'B00137OBEA': (4.666666666666667, 5.0), 'B000VRPGS8': (4.7, 4.128877049712486), 'B000TE2NSY': (5.0, 2.7628865979381443), 'B001NTZFNY': (4.285714285714286, 3.1849277743464253), 'B000W00IGY': (4.5, 4.152345334652093), 'B00L39GAY2': (4.333333333333333, 2.7628865979381443), 'B00136NE8A': (4.9, 4.0), 'B001NTWIN4': (4.5, 2.7628865979381443), 'B013MBXATG': (5.0, 2.7628865979381443), 'B00PMMHEQO': (5.0, 2.7628865979381443), 'B00137IKP6': (4.666666666666667, 4.073962820606583), 'B000W05EGI': (4.7, 5.0), 'B000W020TW': (5.0, 5.0), 'B0

In [ ]:
def cosine_sim_test(ratings_dict, user, other_user):
    cosine_sim_val = {}

    print('User Ratings Dict: ')
    print(ratings_dict[user])
    print('---------------------------')
    print('Other_User: ')
    print(ratings_dict[other_user])
    print('-------------------------')
    dot_prod = sum(ratings_dict[user][item]*ratings_dict[other_user].get(item, 0) for item in ratings_dict[user])
    user_norm = np.linalg.norm(np.array([rating for rating in ratings_dict[user].values()]))
    other_user_norm = np.linalg.norm(np.array([rating for rating in ratings_dict[other_user].values()]))
    norm_mul = user_norm * other_user_norm
    if norm_mul == 0:
        cos_sim = 0
    else:
        cos_sim = dot_prod / norm_mul
    if math.isnan(cos_sim):
        cos_sim = 0
    cosine_sim_val[other_user] = cos_sim

    if cosine_sim_val[other_user] > 1:
        cosine_sim_val[other_user] = 1
    if cosine_sim_val[other_user] < 0:

        print('Sim < 0')
        print('User: ' + str(user))
        print('Other_user' + str(other_user))
        print('Dot Prod: ' + str(dot_prod))
        print('User Norm: ' + str(user_norm))
        print('Other User Norm: ' + str(other_user_norm))
        print('Norm_Mul: ' + str(norm_mul))
        print('Sim value: ' + str(cosine_sim_val[other_user]))
        cosine_sim_val[other_user] = 0        
    
    return cosine_sim_val

In [ ]:
our_user = 'A12R54MKO17TW0'
test_user = 'A1BVU7F2T8EUKS'

In [ ]:
cosine_sim_test(training_dict, our_user, test_user)

In [ ]:
cos_test = cosine_sim(training_dict, our_user)

In [ ]:
sorted(cos_test.items(), key=lambda x: x[1], reverse = True)